In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWhxUs9ekMBYqQ0S5rpimdZCEY-8QA8PJR_FhqVD6bVimQgjoOx0gUg

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
import geemap
import geopandas

In [4]:
## specify areas of interest / districts
#Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
#Districts = ee.FeatureCollection('users/emackres/Wards/Kigali_Sectors_2012_PrincipleComponents')
#Districts = ee.FeatureCollection('users/emackres/KingCountyJurisdictions')
#UrbanShiftCities = ee.FeatureCollection('users/emackres/UrbanShift_cities')
#Districts= UrbanShiftCities.filter(ee.Filter.inList('id',['CRI-San_Jose'])) 
Districts = ee.FeatureCollection('users/emackres/Wards/San_Jose_municipalities')

#print(Districts.limit(1).getInfo())

In [5]:
## create map
Map = geemap.Map(height="350px")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=10)

In [7]:
##Add Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();
Map.addLayer(WorldCover, {'bands': "Map"}, "WorldCover 10m 2020 (ESA)",1);

Map.add_legend(builtin_legend='ESA_WorldCover',position='bottomleft')

In [8]:
Map.addLayer(Districts,{},"Districts")

In [9]:
# get bounding box for Districts area

bb = Districts.union().geometry().bounds(1000)
print(bb.getInfo())
#Map.addLayer(bb,{},"Bounding box")

# get N, S, E, W coordinates from bounding box
# from https://gis.stackexchange.com/questions/318959/get-lon-lat-of-a-top-left-corner-for-geometry-in-google-earth-engine

## return the list of coordinates
listCoords = ee.Array.cat(bb.coordinates(), 1); 
##Casting it to an array makes it possible to slice out the x and y coordinates:
##get the X-coordinates
xCoords = listCoords.slice(1, 0, 1); #print('xCoords', xCoords.getInfo());
yCoords = listCoords.slice(1, 1, 2); #print('yCoords', yCoords.getInfo());

## Reducing each array reveals then the min and max values:
## reduce the arrays to find the max (or min) value
West = ee.Number(xCoords.reduce('min', [0]).get([0,0])).getInfo(); print('West',West);
East = ee.Number(xCoords.reduce('max', [0]).get([0,0])).getInfo(); print('East',East);
North = ee.Number(yCoords.reduce('min', [0]).get([0,0])).getInfo(); print('North',North);
South = ee.Number(yCoords.reduce('max', [0]).get([0,0])).getInfo(); print('South',South);

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-84.46888054107754, 9.551113504203316], [-83.64649300414285, 9.551113504203316], [-83.64649300414285, 10.413695383547827], [-84.46888054107754, 10.413695383547827], [-84.46888054107754, 9.551113504203316]]]}
West -84.46888054107754
East -83.64649300414285
North 9.551113504203316
South 10.413695383547827


In [10]:
# use bounding box to get geodataframe of all OSM data on recreation sites/parks. Use .drop to remove properties that have incompatible names with FeatureCollections

tags = {'leisure':['park','nature_reserve','common','playground']} #{'leisure':['park','nature_reserve','common','pitch','playground','track'],'boundary':['protected_area','national_park']} #}
RecSites = geemap.osm_gdf_from_bbox(North, South, East, West, tags)#.drop(columns=['contact:p.o.box']) # use to remove properties that have incompatible names with FeatureCollections
#RecSites = geemap.osm_gdf_from_bbox(9.09932636176124, 8.833260675723484, 38.90627204838861, 38.63876608591662, {'leisure':'park'}).drop(columns=['contact:p.o.box'])
RecSites = RecSites.reset_index()
print(RecSites.shape[0])
RecSites.sample(3)


1985


,element_type,osmid,leisure,source,geometry,name,addr:city,contact:phone,opening_hours,operator,...,leaf_type,operator:type,boundary,material,place,man_made,height,protect_class,ref:CR:SINAC,start_date
1974,way,1046371686,playground,NaN,"POLYGON ((-84.00292 9.95366, -84.00308 9.95356...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
892,way,130098464,park,KG Ground Survey 2015,"POLYGON ((-84.12873 9.98952, -84.12909 9.98998...",Parque La Lilliana 2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617,way,713693643,park,NaN,"POLYGON ((-84.08185 9.94393, -84.08184 9.94406...",Parque Florida Norte 2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# convert Geodataframe to GeoJson
RecSitesGJ = geemap.gdf_to_geojson(RecSites)

In [12]:
# convert GeoJson to ee.FeatureCollection
RecSitesFC = geemap.geojson_to_ee(RecSitesGJ)#.filter(ee.Filter.eq('element_type','way')).select(['osmid','element_type','geometry','leisure','boundary'])
#print(RecSitesFC.first().toDictionary().getInfo())


In [13]:
# optional and not working in a helpful way yet. 
# simplify geometries and reduce number of properties.  This doesn't seem to reduce size of large FCs sufficiently for the next operations. 

def simplifyFC(feat):
    feat = ee.Feature(feat)
    return feat.simplify(100)#.select(['osmid','element_type','geometry','leisure','boundary'])

RecSitesFCsimp = RecSitesFC.map(simplifyFC)  #.filter(ee.Filter.eq('element_type','way'))
#print(RecSitesFCsimp.first().getInfo())

In [14]:
# add recreation sites to map 
Map.addLayer(RecSitesFC, {}, 'Recreation Sites')
#Map.centerObject(RecSitesFC, 11)
Map

# Large number of or complicated features may lead to "EEException: Request payload size exceeds the limit:" errors when trying to add to map or do additional operations.  Need to simplify geometries in those cases?  Split FC into multiple?  

Map(center=[9.929535560132063, -84.03209086625435], controls=(WidgetControl(options=['position', 'transparent_…

In [15]:
## Make an image, with the same projection as WorldCover, out of the OSM ways in the FC.

WCprojection = WorldCover.projection(); 
#RecSitesImg = RecSitesFC.filter(ee.Filter.eq('element_type','way')).reduceToImage(properties=['osmid'],reducer=ee.Reducer.first());

RecSitesImg = RecSitesFCsimp.style(
  color='gray',
).reproject(
      crs= WCprojection
    )
#print(RecSitesImg.getInfo())

## Display image.
#Map.addLayer(RecSitesImg, {},"Rec Sites Image");

In [16]:
# create image with two bands: BuiltupPixels and UrbanOpenPixels
Builtup = WorldCover.updateMask(WorldCover.eq(50)).rename("BuiltupPixels")
UrbanOpen = RecSitesImg.updateMask(WorldCover.eq(50)).select(1).rename("UrbanOpenPixels")

#Map.addLayer(Builtup, {},"Builtup");
#Map.addLayer(UrbanOpen, {},"UrbanOpen");

comb = Builtup.addBands([UrbanOpen])
#print(comb.getInfo)

In [17]:
## create FeatureCollection with pixels counts of Builtup and UrbanOpen for each feature

OpenBuiltcount=comb.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= Districts, 
  scale= 10, 
  tileScale= 1
)

#print(OpenBuiltcount.limit(1).getInfo())

In [18]:
# Function to translate pixel counts into area and percents

def CountToArea(feat):
    feat = ee.Feature(feat)
    area = ee.Number(feat.area()).multiply(0.000001)
    UrbanOpenArea = ee.Number(feat.getNumber('UrbanOpenPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    BuiltupArea = ee.Number(feat.getNumber('BuiltupPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    OpenAreaPctofBuiltUpArea = ee.Number(UrbanOpenArea).divide(ee.Number(BuiltupArea))

    return feat.set({
        'TotalareaKM2': area,
        'UrbanOpenAreaKM2': UrbanOpenArea,
        'BuiltupAreaKM2': BuiltupArea,
        'OpenAreaPctofBuiltUpArea': OpenAreaPctofBuiltUpArea
    })

# apply function to FeatureCollection
OpenBuiltAreaPct = OpenBuiltcount.map(CountToArea)
#print(OpenBuiltAreaPct.limit(1).getInfo())

In [19]:
Tpctfills = ee.Image().paint(**{'featureCollection': OpenBuiltAreaPct,'color': 'OpenAreaPctofBuiltUpArea'})

fillspalette = ['red', 'green']
Map.addLayer(Tpctfills, {'palette': fillspalette,'min':0,'max':0.1}, 'Open Area as % of built-up area', True, 0.65)
Map

Map(center=[9.929535560132063, -84.03209086625435], controls=(WidgetControl(options=['position', 'transparent_…

In [20]:
# Download OpenBuiltAreaPct as shapefile through URL

# function for converting GeometryCollection to Polygon/MultiPolygon
def filter_polygons(ftr):
    geometries = ftr.geometry().geometries()
    geometries = geometries.map(
        lambda geo: ee.Feature(ee.Geometry(geo)).set('geoType', ee.Geometry(geo).type())
    )

    polygons = (
        ee.FeatureCollection(geometries)
        .filter(ee.Filter.eq('geoType', 'Polygon'))
        .geometry()
    )
    return ee.Feature(polygons).copyProperties(ftr)

OpenBuiltAreaPctSHP = OpenBuiltAreaPct.map(filter_polygons)

col_names = OpenBuiltAreaPctSHP.first().propertyNames().getInfo()
print("Column names: ", col_names)

url = OpenBuiltAreaPctSHP.getDownloadURL("shp", col_names, 'OpenBuiltAreaPctSHP')
print(url)


C:\Users\Eric.Mackres.INTERNAL\Anaconda3\envs\geemap2\lib\site-packages\ee\apifunction.py:206: DeprecationWarning: Feature.geometry() is deprecated: Use Element.geometry()
  bound_function = MakeBoundFunction(api_func)
C:\Users\Eric.Mackres.INTERNAL\Anaconda3\envs\geemap2\lib\site-packages\ee\apifunction.py:206: DeprecationWarning: Feature.geometry() is deprecated: Use Element.geometry()
  bound_function = MakeBoundFunction(api_func)
C:\Users\Eric.Mackres.INTERNAL\Anaconda3\envs\geemap2\lib\site-packages\ee\apifunction.py:206: DeprecationWarning: Feature.geometry() is deprecated: Use Element.geometry()
  bound_function = MakeBoundFunction(api_func)


Column names:  ['system:index', 'fid', 'UrbanOpenAreaKM2', 'shapeID', 'UrbanOpenPixels', 'ObjectId', 'TotalareaKM2', 'OpenAreaPctofBuiltUpArea', 'BuiltupAreaKM2', 'SHAPE_Leng', 'shapeName', 'BuiltupPixels', 'SHAPE_Area']
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/950103076dd415cb4449000fa04d71db-00021eb774503efac784091049057ce0:getFeatures


In [21]:
# Download FeatureCollection of OpenBuiltAreaPct as shapefile to Google Drive

# Set configuration parameters for output vector
task_config = {
    #'folder': 'gee-data',  # output Google Drive folder
    'fileFormat': 'SHP',
    #'selectors': col_names,  # a list of properties/attributes to be exported
}
#print('Exporting {}'.format(OpenBuiltAreaPct))
task = ee.batch.Export.table.toDrive(OpenBuiltAreaPct, 'OpenBuiltAreaPct', **task_config)
task.start()

In [22]:

# Download FeatureCollection of OSM data (ways/polygons only) to Google Drive as geojson
# Set configuration parameters for output vector
task_config = {
    #'folder': 'gee-data',  # output Google Drive folder
    'fileFormat': 'Geojson',
    #'selectors': col_names,  # a list of properties/attributes to be exported
}

#print('Exporting {}'.format(RecSitesFC))
task = ee.batch.Export.table.toDrive(RecSitesFC, 'RecSites', **task_config)
task.start()